In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import pickle
import pandas as pd

import statsmodels.stats.api as sms

In [3]:
with open("mldb_2021-07-16.pickle", 'rb') as f:
    evals = pickle.load(f)

In [4]:
df = pd.DataFrame(evals)

In [5]:
# Ignore checkpoints for now
df = df[df.epoch == -1]

In [6]:
# Add metrics
df['accuracy'] = df.stats.apply(lambda x: x['num_correct_and_size'][0] / x['num_correct_and_size'][1])

num_correct_vals = df.stats.apply(lambda x: x['num_correct_and_size'][0]).values
size_vals = df.stats.apply(lambda x: x['num_correct_and_size'][1]).values
cis = sms.proportion_confint(num_correct_vals, size_vals, alpha=0.05, method='beta')
df['accuracy_ci'] = list(zip(*cis))

In [7]:
# Pairs of id-train, id-test
ID_PAIRS = [
    ("cifar10-train", "cifar10-test"),
    ("FMoW-train", "FMoW-id_test"),
    ("FMoW-train", "FMoW-id_val"),
    # TODO: Add more here
]

In [8]:
def reformat(_df, train, test):
    test_eval = _df[_df.test_set == test]
    if len(test_eval) == 0:
        return pd.DataFrame()
    test_eval = test_eval.iloc[0]
    shift_evals = _df[~_df.test_set.isin([train, test])]
    newdf = shift_evals[["model_family", "model_id", "epoch", "rule_params"]]
    newdf = newdf.rename(columns={"rule_params": "hyperparameters"})
    newdf["train_set"] = train
    newdf["test_set"] = test
    newdf["shift_set"] = shift_evals["test_set"]
    newdf["test_accuracy"] = test_eval.accuracy
    newdf["test_accuracy_ci"] = [test_eval.accuracy_ci for _ in range(len(newdf))]
    newdf["shift_accuracy"] = shift_evals["accuracy"]
    newdf["shift_accuracy_ci"] = shift_evals["accuracy_ci"]
    return newdf

In [9]:
new_df = []
for train, test in ID_PAIRS:
    df_train = df[df.train_set == train]
    shift_sets = set(df_train.test_set) - set([train, test])
    new_df.extend([reformat(modeldf, train, test) for _, modeldf in df_train.groupby("model_id")])

In [10]:
new_df = pd.concat(new_df)

In [11]:
def rename_model_family(model_family):
    if "RandFeatures" in model_family:
        return "RandomFeatures"
    elif "K_nearest_neighbors" in model_family:
        return "KNN"
    return model_family

new_df["model_family"] = new_df.model_family.apply(rename_model_family)

In [22]:
# Filter out random shift sets
def shift_filter(row):
    if row.train_set == "cifar10-train":
        shift_set = row.shift_set
        if "cifar10c" in shift_set:
            return True
        if "CifarCov" in shift_set:
            return False
        if "tmp" in shift_set:
            return False
        if "train_10k" in shift_set:
            return False
        return True
    return True

filtered_df = new_df[new_df.apply(shift_filter, axis=1)]

In [23]:
filtered_df.to_csv("results.csv", index=False)

In [30]:
a = sorted(filtered_df.train_set.unique(), ascending=False)

TypeError: 'ascending' is an invalid keyword argument for sort()

In [29]:
a.index("yo")

ValueError: 'yo' is not in list